# Singular Value Decomposition(SVD) Matrix Factorization 

# Importing Dependencies

In [1]:
from MovieLens import MovieLens
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

## Loading Data

In [2]:
def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

In [3]:
np.random.seed(0)
random.seed(0)

In [4]:
# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...


## Initialising Evaluator

In [5]:
# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


## Implementing SVD using SupriseLib

In [6]:
# SVD
SVD = SVD()
evaluator.AddAlgorithm(SVD, "SVD")

In [7]:
# SVD++
SVDPlusPlus = SVDpp()
evaluator.AddAlgorithm(SVDPlusPlus, "SVD++")

In [8]:
# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

## Evaluate

In [9]:
evaluator.Evaluate(False)

Evaluating  SVD ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD++ ...
Evaluating accuracy...
Analysis complete.
Evaluating  Random ...
Evaluating accuracy...
Analysis complete.


Algorithm  RMSE       MAE       
SVD        0.9039     0.6984    
SVD++      0.8943     0.6887    
Random     1.4359     1.1493    

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.


In [10]:
evaluator.SampleTopNRecs(ml)


Using recommender  SVD

Building recommendation model...
Computing recommendations...

We recommend:
Gladiator (1992) 4.520884890007874
Philadelphia Story, The (1940) 4.420701711947352
Stand by Me (1986) 4.3959589752178365
Moon (2009) 4.372613693384055
Happiness (1998) 4.369493252705134
American Graffiti (1973) 4.353470600109924
And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001) 4.349192492630499
Wallace & Gromit: A Close Shave (1995) 4.3154412154304085
Band of Brothers (2001) 4.315414828016616
Seven Samurai (Shichinin no samurai) (1954) 4.311102920673881

Using recommender  SVD++

Building recommendation model...
Computing recommendations...

We recommend:
Lock, Stock & Two Smoking Barrels (1998) 4.60422766627621
The Imitation Game (2014) 4.457817607681914
Amadeus (1984) 4.322529285260794
Indiana Jones and the Last Crusade (1989) 4.289355864317423
Happiness (1998) 4.260333724293291
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001) 4.22951198

## Tuning SVD

In [11]:
del SVD

In [12]:
from surprise import SVD

## GridSearchCV

In [13]:
from surprise.model_selection import GridSearchCV

In [14]:
print("Searching for best parameters...")
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
              'n_factors': [50, 100]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(evaluationData)

Searching for best parameters...


In [15]:
# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])

Best RMSE score attained:  0.8998197388777108


In [16]:
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

{'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50}


## Intialising the evaluator

In [17]:
# Construct an Evaluator to, you know, evaluate them
from Evaluator import Evaluator
evaluator = Evaluator(evaluationData, rankings)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [18]:
params = gs.best_params['rmse']
SVDtuned = SVD(n_epochs = params['n_epochs'], lr_all = params['lr_all'], n_factors = params['n_factors'])
evaluator.AddAlgorithm(SVDtuned, "SVD - Tuned")

In [19]:
SVDUntuned = SVD()
evaluator.AddAlgorithm(SVDUntuned, "SVD - Untuned")

In [20]:
# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

In [21]:
evaluator.Evaluate(False)

Evaluating  SVD ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD++ ...
Evaluating accuracy...
Analysis complete.
Evaluating  Random ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD - Tuned ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD - Untuned ...
Evaluating accuracy...
Analysis complete.
Evaluating  Random ...
Evaluating accuracy...
Analysis complete.


Algorithm  RMSE       MAE       
SVD        0.9048     0.7010    
SVD++      0.8970     0.6900    
Random     1.4427     1.1512    
SVD - Tuned 0.9019     0.6979    
SVD - Untuned 0.9030     0.6993    

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.


In [22]:
evaluator.SampleTopNRecs(ml)


Using recommender  SVD

Building recommendation model...
Computing recommendations...

We recommend:
African Queen, The (1951) 4.770854516835359
Lord of the Rings: The Return of the King, The (2003) 4.638142543075613
Godfather, The (1972) 4.504155895826655
Lord of the Rings: The Fellowship of the Ring, The (2001) 4.502097739458008
Seven Samurai (Shichinin no samurai) (1954) 4.485480278997229
Butch Cassidy and the Sundance Kid (1969) 4.433675291981523
Lord of the Rings: The Two Towers, The (2002) 4.4166478939536455
Lock, Stock & Two Smoking Barrels (1998) 4.409013100897731
It Happened One Night (1934) 4.379342254424493
Godfather: Part II, The (1974) 4.363521816740054

Using recommender  SVD++

Building recommendation model...
Computing recommendations...

We recommend:
Lord of the Rings: The Fellowship of the Ring, The (2001) 4.703082557525876
Lord of the Rings: The Two Towers, The (2002) 4.557029405027646
Matrix, The (1999) 4.448549575900148
Saving Private Ryan (1998) 4.42079922519089